In [1]:
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext
from bigdl.util.common import *


# create sparkcontext with bigdl configuration
sc = SparkContext.getOrCreate(conf=create_spark_conf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession(sparkContext = sc)
spark.conf.set("spark.sql.session.timeZone", "UTC")

from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.util.common import *

import numpy as np
import time

In [2]:
learning_rate = 0.1
training_epochs = 100
batch_size = 10
init_engine()

In [3]:
#Load the data
df = spark.read.csv(f"hdfs://localhost:9000/solar_data/processed/Arkansas_1.csv", inferSchema=True, header=True)

In [12]:
cols = df.columns
cols.remove('year')
cols.remove('month')
cols.remove('day')
cols.remove('sum(GHI)')

In [44]:
ys = df.select('sum(GHI)')
Xs = df.select(cols)

#converting to rdd
rddys = ys.rdd.map(list)
rddXs = Xs.rdd.map(list)

X_train, X_test = rddXs.randomSplit([.8, .2])
y_train, y_test = rddys.randomSplit([.8, .2])

In [45]:
len(Xs.columns)

168

In [46]:
len(rddXs.take(1)[0])

168

In [48]:
def build_model(input_size, hidden_size, output_size):
    model = Sequential()
    recurrent = Recurrent()
    recurrent.add(LSTM(input_size, hidden_size))
    model.add(InferReshape([-1, input_size], True))
    model.add(recurrent)
    model.add(Select(2, -1))
    model.add(Linear(hidden_size, output_size))
    return model

In [50]:
model = build_model(168,50,1)

creating: createSequential
creating: createRecurrent
creating: createTanh
creating: createSigmoid
creating: createLSTM
creating: createInferReshape
creating: createSelect
creating: createLinear


In [51]:
# Create an Optimizer
optimizer = Optimizer(
    model=model,
    training_rdd=X_train,
    criterion=MSECriterion(),
    optim_method=SGD(learningrate=learning_rate),
    end_trigger=MaxEpoch(training_epochs),
    batch_size=batch_size)

creating: createMSECriterion
creating: createDefault
creating: createSGD
creating: createMaxEpoch
creating: createDistriOptimizer


/home/dyllanjr/anaconda3/envs/solarenv/lib/python3.6/site-packages/bigdl/optim/optimizer.py:864: UserWarning: You are recommended to use `create` method to create an optimizer.
  warnings.warn("You are recommended to use `create` method to create an optimizer.")


In [ ]:
model.predict()

In [52]:
#Predict sequence of 50 steps before shifting prediction run forward by 50 steps
prediction_seqs = []
data = X_test
prediction_len = 1
window_size=1
for i in range(data.count()//prediction_len):
    curr_frame = data[i*int(prediction_len)]
    predicted = []
    for j in range(int(prediction_len)):

        curr_data = curr_frame[np.newaxis,:,:]

        curr_data_rdd = sc.parallelize(curr_data).map(lambda x: Sample.from_ndarray(x,[0]))

        preds = model.predict(curr_data_rdd).collect()[0][0][0]
        predicted.append(preds)
        curr_frame = curr_frame[1:]
        curr_frame = np.insert(curr_frame, [window_size-1], predicted[-1], axis=0)
        prediction_seqs.append(predicted)
predictions = prediction_seqs

TypeError: 'PipelinedRDD' object does not support indexing